In [1]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.4MB 4.4MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 
     |████████████████████████████████| 112kB 23.0MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73604 sha256=7dd2bed048b80fefd719f03ecf7eb170ebdf1d4768dfe4d450f290a288ae33b6
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=34850726f8c19945a1c2dea20ffd9d763fdff70cdbe5fbd455b7fbc46f9f1300
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=677b5e4bf9d5655b69b125aaafde45b82f47c4642e041cd128bb24657e31bb9f
  Stored in directory: /root/.cache/pip/wheels/ce

In [2]:
!wandb login ee9416edde558c322450d0ec80266d2c0db81f45

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [3]:
!pip install nlpaug

     |████████████████████████████████| 102kB 3.5MB/s 


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import wandb

import os
import zipfile
for dirname, _, filenames in os.walk('/kaggle/input/quora-insincere-questions-classification/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import time
import os
import pandas as pd
import numpy as np
import random
import copy
import scipy

import re
from gensim.models import KeyedVectors
import gc

from keras.preprocessing.text import Tokenizer
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

t0 = time.time()

data_path = '/kaggle/input/quora-insincere-questions-classification/'
seed = 2077

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv


Using TensorFlow backend.


/kaggle/input/paragram-300-sl999/paragram_300_sl999.txt
/kaggle/input/augmented/augmented.csv
/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/augmented-fairseq/augmented (1).csv
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv


In [5]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb
/kaggle/working/wandb/settings


## Constants

In [6]:
swap_prob = 0.5
# Preprocessing
max_len = 50
lower = True
trunc = 'pre'
max_features = 120000
n_vocab = max_features
clean_num = 0

# Training
aug_epochs = 5
batch_size = 512
drop_last = True
dropout = 0.35

hidden_dim = 128

# Embedding
fix_embedding = True
unk_uni = True  # Initializer for unknown words делает вектор для незнакомого слова из N(emb.mean(), emb.std())
n_embed = 2
embed_dim = n_embed * 300 
proj_dim = hidden_dim

# GRU
bidirectional = True
n_layers = 1
rnn_dim = hidden_dim

# The second last Linear layer
dense_dim = 2 * rnn_dim if bidirectional else rnn_dim

# Test set
test_batch_size = 8 * batch_size

def seed_torch(seed=1):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_torch(seed)
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'


def get_param_size(model, trainable=True):
    if trainable:
        psize = np.sum([np.prod(p.size()) for p in model.parameters() if p.requires_grad])
    else:
        psize = np.sum([np.prod(p.size()) for p in model.parameters()])
    return psize

> ## GRU Model

In [7]:
class GlobalMaxPooling1D(nn.Module):
    def __init__(self):
        super(GlobalMaxPooling1D, self).__init__()

    def forward(self, inputs):
        z, _ = torch.max(inputs, 1)
        return z

    def __repr__(self):
        return self.__class__.__name__ + '()'


class GRUModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, proj_dim, rnn_dim, n_layers, bidirectional, dense_dim, dropout=0.2,
                 padding_idx=0, pretrained_embedding=None, fix_embedding=True,
                 n_out=1):
        super(GRUModel, self).__init__()
        self.n_vocab = n_vocab
        self.embed_dim = embed_dim
        self.n_layers = n_layers
        self.dense_dim = dense_dim
        self.n_out = n_out
        self.bidirectional = bidirectional
        self.fix_embedding = fix_embedding
        self.padding_idx = padding_idx
        if pretrained_embedding is not None:
            self.embed = nn.Embedding.from_pretrained(pretrained_embedding, freeze=fix_embedding)
            self.embed.padding_idx = self.padding_idx
        else:
            self.embed = nn.Embedding(self.n_vocab, self.embed_dim, padding_idx=self.padding_idx)
        self.embed_drop = nn.Dropout(dropout + 0.1)
        self.linear_drop1 = nn.Dropout(dropout)
        self.linear_drop2 = nn.Dropout(dropout)
        self.gru = nn.GRU(embed_dim, rnn_dim, self.n_layers,
                          batch_first=True, bidirectional=bidirectional, dropout=dropout)
        self.pooling = GlobalMaxPooling1D()
        in_dim = 2 * rnn_dim if self.bidirectional else rnn_dim
        self.dense = nn.Linear(in_dim, dense_dim)
        self.dense_act = nn.ReLU()
        self.batchnorm = nn.BatchNorm1d(dense_dim)
        self.out_linear = nn.Linear(dense_dim, n_out)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if name.find('embed') > -1:
                continue
            elif name.find('weight') > -1 and len(param.size()) > 1:
                nn.init.xavier_uniform_(param)

    def forward(self, inputs):
        # inputs: (bs, max_len)
        x = self.embed_drop(self.embed(inputs))
        x, hidden = self.gru(x)
        x = self.pooling(x)
        x = self.linear_drop1(x)
        x = self.dense_act(self.batchnorm(self.dense(x)))
        x = self.linear_drop2(x)
        x = self.out_linear(x)
        return x

    def fit(self, dataloader, valloader, epochs, optimizer, scheduler, val_ans, patience=3, callbacks=None):
        early_stopping = EarlyStopping(patience=patience, verbose=True)
        for i in range(epochs):
            print('epoch: ', i + 1)
            self.run_epoch(dataloader, valloader, optimizer, scheduler, val_ans, early_stopping, callbacks=callbacks)
            if early_stopping.early_stop:
                print("Early stopping")
                break

    def predict(self, dataloader):
        preds = []
        with torch.no_grad():
            for batch in dataloader:
                batch = tuple(t.to(device) for t in batch)
                X_batch = batch[0]
                preds.append(self.forward(X_batch).data.cpu())
        return torch.cat(preds)

    def predict_proba(self, dataloader):
        return torch.sigmoid(self.predict(dataloader)).data.numpy()
    
    def run_epoch(self, dataloader, valloader, optimizer, scheduler, val_ans, early_stopping, callbacks=None,
              criterion=nn.BCEWithLogitsLoss()):
        t1 = time.time()
        tr_loss = 0
        tr_roc_auc = 0
        self.train()
        for step, batch in enumerate(dataloader):
            batch = tuple(t.to(device) for t in batch)
            x_batch, y_batch = batch
            optimizer.zero_grad()
            outputs = self.forward(x_batch)
            loss = criterion(outputs[:, 0], y_batch.float())
            loss.backward()
            optimizer.step()
            tr_loss += loss.item()
            tr_roc_auc += roc_auc_score(y_batch.cpu(), torch.sigmoid(outputs).data.cpu().numpy())
            if callbacks is not None:
                for func in callbacks:
                    func.on_batch_end(self)
        tr_roc_auc /= step
        tr_loss /= step
        self.eval()
        val_preds = self.predict(valloader)
        val_ans = torch.tensor(val_ans.values, dtype=torch.float64)
        val_loss = criterion(val_preds[:, 0], val_ans).item()
        val_roc_auc = roc_auc_score(val_ans.int().cpu(), torch.sigmoid(val_preds).data.cpu().numpy())
        print(f'val loss:{val_loss:.7f}')
        print(f'train loss:{tr_loss:.7f}')
        early_stopping(val_loss, self)    
        if callbacks is not None:
            for func in callbacks:
                func.on_epoch_end(self)
        wandb.log({"loss/train": tr_loss})
        wandb.log({"loss/val": val_loss})
        wandb.log({"roc_auc/train": tr_roc_auc})
        wandb.log({"roc_auc/val": val_roc_auc})
        scheduler.step(val_loss)
        return tr_loss

In [8]:
#https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [9]:
class TextClassificationDataset(Dataset):
    """
    token_ids_s : List of token ids
    labels   : Target labels
    training: Sort token_ids_s by length if training is True.
    """
    def __init__(self, token_ids_s, labels=None, max_len=1000, training=True, sort=True):
        self.training = training

        if labels is None:
            self.labels = torch.ones(len(token_ids_s), dtype=torch.long)  # dummy
        else:
            self.labels = torch.LongTensor(labels)

        seq_lens = []
        self.inputs = []
        for e, token_ids in enumerate(token_ids_s):
            seq_lens.append(len(token_ids))
            input_ids = torch.LongTensor(token_ids[:max_len])
            self.inputs.append(input_ids)

        if self.training and sort:
            self.indices = np.argsort(seq_lens)
            self.inputs = [self.inputs[i] for i in self.indices]
            self.labels = self.labels[self.indices]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

    def set_labels(self, labels):
        self.labels = torch.tensor(labels, dtype=torch.long)


# Always drop last
class BatchIterator(object):

    def __init__(self, dataset, collate_fn, batch_size,
                 shuffle=True, drop_last=True):
        self.dataset = dataset
        self.collate_fn = collate_fn
        self.batch_size = batch_size
        self.size = len(dataset)
        self.shuffle = shuffle
        if drop_last:
            self.num_batches = self.size // batch_size
        else:
            self.num_batches = (self.size + self.batch_size - 1) // self.batch_size

    def __iter__(self):
        if self.shuffle:
            indices = np.random.choice(self.num_batches, self.num_batches, replace=False)
        else:
            indices = range(self.num_batches)
        for idx in indices:
            left = self.batch_size * idx
            yield(self.collate_fn(self.dataset[left: left + self.batch_size]))

    def __len__(self):
        return self.num_batches


def collate_fn(batch):
    xy_batch = [pad_sequence(batch[0], batch_first=True), batch[1]]
    return xy_batch

## Data preprocessing

In [10]:
puncts = ',.":)(-!?|;\'$&/[]>%=#*+\\•~@£·_{}©^®`<→°€™›♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║\
―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'


def clean_text(x, puncts=puncts): #добавляет пробелы вокруг пунктуации
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def prepare_data(train_df, val_df, test_df, max_len, max_features, trunc='pre',
                 lower=False, clean_num=2, unk_uni=True):
    train_df = train_df.copy()
    train_y = train_df['target'].values


    # fill up the missing values
    train_df['text'] = train_df['text'].fillna('_##_')
    val_df['text'] = val_df['text'].fillna('_##_')
    test_df['text'] = test_df['text'].fillna('_##_')

    train_X = train_df['text'].values
    val_X = val_df['text'].values
    test_X = test_df['text'].values

    tokenizer = Tokenizer(num_words=max_features, lower=True, filters='')
    tokenizer.fit_on_texts(train_X)
    print(tokenizer)
    print('len(vocab)', len(tokenizer.word_index))

    train_X_ids = tokenizer.texts_to_sequences(train_X)
    train_dataset = TextClassificationDataset(train_X_ids, train_y, sort=False)
    train_loader = BatchIterator(train_dataset, collate_fn, batch_size)
    
    val_X_ids = tokenizer.texts_to_sequences(val_X)
    val_dataset = TextClassificationDataset(val_X_ids, training=False)
    val_loader = BatchIterator(val_dataset, collate_fn, batch_size, shuffle=False, drop_last=False)


    test_X_ids = tokenizer.texts_to_sequences(test_X)
    test_dataset = TextClassificationDataset(test_X_ids, training=False)
    test_loader = BatchIterator(test_dataset, collate_fn, batch_size, shuffle=False, drop_last=False)
    return train_loader, val_loader, test_loader, tokenizer

## Data loading

In [11]:
train_df = pd.read_csv(data_path  + 'train.csv')

In [12]:
def make_augmented(p=0.35):
    aug = nac.RandomCharAug(action='swap', aug_word_p=p)
    augmented_list = aug.augments(oneline['text'].values)
    augmented = pd.DataFrame({'text' : augmented_list, 
                          'target' : list(oneline['target'].values)})
    return augmented


In [13]:
train_df = train_df.rename(columns={'question_text': 'text'})

train_df['text'] = train_df['text'].apply(str.lower)
train_df['text'] = train_df['text'].apply(clean_text)
train_df['text'] = train_df['text'].apply(clean_numbers)

In [14]:
train_df, test_df = train_test_split(train_df.copy(), train_size=0.7, random_state=seed)
oneline = train_df[~train_df['text'].str.contains('\n')]
multiline = train_df[train_df['text'].str.contains('\n')]
oneline_size = len(oneline)
train_idx, valid_idx = train_test_split(np.arange(oneline_size), train_size=0.9, random_state=seed)

## Model training

In [15]:
def train(aug_prob, epochs):
    wandb.init(project="trained embeddings", name=f'char swap {aug_prob}',reinit=True)
    #wandb.init(project="single model", name=f'trained char swap {aug_prob}',reinit=True)
    
    augmented = make_augmented(aug_prob)
    
    augmented['text'] = augmented['text'].apply(str.lower)
    augmented['text'] = augmented['text'].apply(clean_text)
    augmented['text'] = augmented['text'].apply(clean_numbers)
    
    
    train_aug = pd.DataFrame({
        'text' : 
        list(oneline.iloc[train_idx]['text'].values) +
        list(augmented.iloc[train_idx]['text'].values) +
        list(multiline['text'].values),
        'target' :
        list(oneline.iloc[train_idx]['target'].values) +
        list(augmented.iloc[train_idx]['target'].values) +
        list(multiline['target'].values)
    })


    valid_df = oneline.iloc[valid_idx]

    train_aug.drop_duplicates(inplace=True)
    train_aug = train_aug.sample(frac=1)
    
    #del augmented
    gc.collect()
    

    data_path = '/kaggle/working/'
    obj = prepare_data(train_aug, valid_df, test_df, max_len, max_features,
                   trunc=trunc, lower=lower, clean_num=clean_num, unk_uni=unk_uni)

    train_loader, val_loader, test_loader, tokenizer = obj

    vocab_size = len(tokenizer.word_index)
    
    word_index = tokenizer.word_index
    
    val_ans, test_ans = valid_df['target'], test_df['target']

    model = GRUModel(n_vocab, embed_dim, proj_dim, rnn_dim, n_layers, bidirectional, dense_dim, padding_idx=0)
    print(model)
    print('#Trainable params', get_param_size(model))
    model.to(device)
    optimizer = Adam(model.parameters(), lr=0.0005)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=2, factor=0.5)
    
    config =  {
            'model': 'bigru',
            'optimizer': str(optimizer),
            'lr' : 0.0005,
            'scheduler': 'plateau',
            'early_stop': 3,
            'vocab_size': n_vocab,
             'h_size': hidden_dim,
             'n_layers': n_layers,
             'dropout': dropout,
             'batch_size': batch_size,
             'embed_dim': embed_dim,
             'max_len': max_len,
             'batch_norm' : 'true'
        }
    wandb.config.update(config)
    
    val_probas = np.zeros((len(valid_df), 1))
    test_probas = np.zeros((len(test_df), 1))

    t2 = time.time()
    wandb.watch(model)
    model.fit(train_loader, val_loader, epochs, optimizer, scheduler, val_ans)
    print(f'n_model:{1} {(time.time() - t2) / (aug_epochs):.1f}s/epoch')
    t3 = time.time()
    val_probas =  model.predict_proba(val_loader)
    test_probas = model.predict_proba(test_loader)
    print(f'n_model:{1} {roc_auc_score(val_ans, val_probas)}')
    print(f'{time.time() - t3:.1f}s')

    res = scipy.optimize.minimize(
        lambda t: -f1_score(val_ans, (val_probas >= t).astype(np.int)),
        x0=0.5,
        method='Nelder-Mead',
        tol=1e-3,
    )
    best_threshold = res.x[0]
    print(best_threshold)
    
    f1_val = f1_score(val_ans, (val_probas >= best_threshold))
    f1_test = f1_score(test_ans, test_probas >= best_threshold)
    wandb.log({'f1/val' : f1_val})
    wandb.log({'f1/test' : f1_test})
    roc_auc = roc_auc_score(test_ans, test_probas)
    print('test f1-score: {}'.format(f1_test))
    print('test roc-auc score: {}'.format(roc_auc))
    
    wandb.join()


In [16]:
augs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
epochs_num = [4, 3, 4, 3, 3, 3]

for aug_p, epochs in zip(augs, epochs_num):
    train(aug_p, epochs)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


len(vocab) 271637


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


GRUModel(
  (embed): Embedding(120000, 600, padding_idx=0)
  (embed_drop): Dropout(p=0.30000000000000004, inplace=False)
  (linear_drop1): Dropout(p=0.2, inplace=False)
  (linear_drop2): Dropout(p=0.2, inplace=False)
  (gru): GRU(600, 128, batch_first=True, dropout=0.2, bidirectional=True)
  (pooling): GlobalMaxPooling1D()
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (dense_act): ReLU()
  (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out_linear): Linear(in_features=256, out_features=1, bias=True)
)
#Trainable params 72627201
epoch:  1
val loss:0.1151465
train loss:0.1381323
Validation loss decreased (inf --> 0.115147).  Saving model ...
epoch:  2
val loss:0.1098086
train loss:0.1109639
Validation loss decreased (0.115147 --> 0.109809).  Saving model ...
epoch:  3
val loss:0.1086720
train loss:0.1020999
Validation loss decreased (0.109809 --> 0.108672).  Saving model ...
epoch:  4
val loss:0.1084213
train loss:0.095

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

GRUModel(
  (embed): Embedding(120000, 600, padding_idx=0)
  (embed_drop): Dropout(p=0.30000000000000004, inplace=False)
  (linear_drop1): Dropout(p=0.2, inplace=False)
  (linear_drop2): Dropout(p=0.2, inplace=False)
  (gru): GRU(600, 128, batch_first=True, dropout=0.2, bidirectional=True)
  (pooling): GlobalMaxPooling1D()
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (dense_act): ReLU()
  (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out_linear): Linear(in_features=256, out_features=1, bias=True)
)
#Trainable params 72627201
epoch:  1
val loss:0.1197253
train loss:0.1479896
Validation loss decreased (inf --> 0.119725).  Saving model ...
epoch:  2
val loss:0.1100247
train loss:0.1174036
Validation loss decreased (0.119725 --> 0.110025).  Saving model ...
epoch:  3
val loss:0.1095976
train loss:0.1072418
Validation loss decreased (0.110025 --> 0.109598).  Saving model ...
n_model:1 89.3s/epoch
n_model:1 0.95765688294

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


len(vocab) 528774


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


GRUModel(
  (embed): Embedding(120000, 600, padding_idx=0)
  (embed_drop): Dropout(p=0.30000000000000004, inplace=False)
  (linear_drop1): Dropout(p=0.2, inplace=False)
  (linear_drop2): Dropout(p=0.2, inplace=False)
  (gru): GRU(600, 128, batch_first=True, dropout=0.2, bidirectional=True)
  (pooling): GlobalMaxPooling1D()
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (dense_act): ReLU()
  (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out_linear): Linear(in_features=256, out_features=1, bias=True)
)
#Trainable params 72627201
epoch:  1
val loss:0.1171138
train loss:0.1474073
Validation loss decreased (inf --> 0.117114).  Saving model ...
epoch:  2
val loss:0.1098082
train loss:0.1185614
Validation loss decreased (0.117114 --> 0.109808).  Saving model ...
epoch:  3
val loss:0.1096317
train loss:0.1087035
Validation loss decreased (0.109808 --> 0.109632).  Saving model ...
n_model:1 88.0s/epoch
n_model:1 0.95726531823